In [1]:
import pandas as pd
import numpy as np

In [2]:
# carregamento dos dados

expressao_df = pd.read_excel("../dados npo/DADOS PUBLICOS/Cópia de Expression_GC.xlsx", index_col=0)
metadata_df = pd.read_excel('/Users/polyanasilva/Documents/workspace/npo/dados npo/DADOS PUBLICOS/Cópia de Clinical_GC.xlsx', index_col=0)


### Inspecao inicial dos dados

In [3]:
print("INSPEÇÃO INICIAL - DADOS DE EXPRESSÃO")
print("=" * 50)
print(f"Shape: {expressao_df.shape}")  # (pacientes, genes)
print(f"Total de valores: {expressao_df.size}")
print(f"Genes: {expressao_df.index[:5].tolist()}...")  # Primeiros 5
print(f"Pacientes: {expressao_df.columns[:5].tolist()}...")   # Primeiros 5 genes

print("\nINSPEÇÃO INICIAL - METADADOS")
print("=" * 50)
print(f"Shape: {metadata_df.shape}")
print(f"Colunas disponíveis: {metadata_df.columns.tolist()}")
print(f"Primeiros pacientes: {metadata_df.index[:5].tolist()}...")

print("\nDistribuição dos subtipos moleculares:")
print("=" * 50)
print(metadata_df["paper_Molecular.Subtype"].value_counts())
print("valores nulos: ", metadata_df["paper_Molecular.Subtype"].isnull().sum())

INSPEÇÃO INICIAL - DADOS DE EXPRESSÃO
Shape: (36940, 412)
Total de valores: 15219280
Genes: ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FIRRM']...
Pacientes: ['TCGA-D7-6815-01A-11R-1884-13', 'TCGA-IN-A7NT-01A-21R-A354-31', 'TCGA-BR-8365-01A-21R-2343-13', 'TCGA-B7-A5TJ-01A-11R-A31P-31', 'TCGA-CD-8535-01A-11R-2343-13']...

INSPEÇÃO INICIAL - METADADOS
Shape: (412, 112)
Colunas disponíveis: ['patient', 'sample', 'shortLetterCode', 'definition', 'sample_submitter_id', 'sample_type_id', 'tumor_descriptor', 'sample_id', 'sample_type', 'composition', 'days_to_collection', 'state', 'initial_weight', 'preservation_method', 'intermediate_dimension', 'pathology_report_uuid', 'submitter_id', 'shortest_dimension', 'oct_embedded', 'specimen_type', 'longest_dimension', 'is_ffpe', 'tissue_type', 'synchronous_malignancy', 'ajcc_pathologic_stage', 'days_to_diagnosis', 'treatments', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'prior_malignancy', 'year_of_diagnosis', 'prior_treatment', 

In [4]:
expressao_df.head()

,TCGA-D7-6815-01A-11R-1884-13,TCGA-IN-A7NT-01A-21R-A354-31,TCGA-BR-8365-01A-21R-2343-13,TCGA-B7-A5TJ-01A-11R-A31P-31,TCGA-CD-8535-01A-11R-2343-13,TCGA-VQ-A8PK-01A-12R-A414-31,TCGA-VQ-AA6D-01A-11R-A414-31,TCGA-CG-5721-01A-11R-1602-13,TCGA-FP-8099-01A-11R-2343-13,TCGA-BR-A4J5-01A-21R-A251-31,...,TCGA-VQ-A91V-01A-11R-A414-31,TCGA-VQ-A91X-01A-12R-A414-31,TCGA-VQ-A923-01A-11R-A414-31,TCGA-EQ-8122-01A-11R-2343-13,TCGA-VQ-A8PO-01A-11R-A414-31,TCGA-B7-A5TK-01A-12R-A36D-31,TCGA-D7-8578-01A-21R-2343-13,TCGA-BR-A4J9-01A-12R-A251-31,TCGA-BR-4366-01A-01R-1157-13,TCGA-VQ-A8PH-01A-12R-A414-31
TSPAN6,3875,1352,751,3338,4901,1174,3838,1950,1943,1728,...,1184,32,1972,2813,4360,5816,2902,693,2560,6469
TNMD,5,0,6,1,0,0,0,4,0,1,...,0,1,0,2,22,3,2,0,6,1
DPM1,2712,1849,1817,3823,6283,2242,4371,1825,3279,2406,...,2479,4732,3558,5673,2483,4611,2763,2393,12395,6627
SCYL3,1265,199,1046,1064,1445,885,1324,553,1136,1069,...,1649,1092,1330,1520,688,1041,768,1352,2650,624
FIRRM,501,264,467,607,1246,848,1034,385,804,366,...,895,765,727,1201,591,449,536,251,1540,707


### Filtragem por baixa expressao ou variância

In [5]:
# FILTRO POR EXP MINIMA

limiar = 1        # valor mínimo após log2
frac_min = 0.8   # gene deve estar expresso em pelo menos 80% das amostras

# Criar máscara booleana ?? pesquisar oq isso
mask = (expressao_df > limiar).sum(axis=1) >= (frac_min * expressao_df.shape[1])

# Filtrar genes
df_filtrado = expressao_df[mask]

print(f"Genes antes do filtro: {expressao_df.shape[0]}")
print(f"Genes depois do filtro: {df_filtrado.shape[0]}")

Genes antes do filtro: 36940
Genes depois do filtro: 20129


In [6]:
# Filtrar por variância

variancias = df_filtrado.var(axis=1)
# sem analise da media

# Filtro baseado em limiares absolutos (dar uma olhada no Guo 2019)
mask = (variancias >= 1.0)

print(f"Genes antes: {df_filtrado.shape[0]}")
df_filtr_novo = df_filtrado[mask]


print(f"Genes depois (variância>=1.0): {df_filtr_novo.shape[0]}")

Genes antes: 20129
Genes depois (variância>=1.0): 20129


In [7]:
variancias_ordenadas = variancias.sort_values(ascending=False)
variancias_ordenadas

COX1        1.025416e+12
ND4         3.680995e+11
COL1A1      2.035156e+11
PGC         2.024109e+11
COX3        1.959166e+11
                ...     
HNRNPCP6    1.300562e+01
SF3B4P1     1.278957e+01
ATP5MGP3    1.236336e+01
RPL31P31    1.234660e+01
HNRNPCL1    1.129625e+01
Length: 20129, dtype: float64

In [8]:
df_filtr_novo.head()

,TCGA-D7-6815-01A-11R-1884-13,TCGA-IN-A7NT-01A-21R-A354-31,TCGA-BR-8365-01A-21R-2343-13,TCGA-B7-A5TJ-01A-11R-A31P-31,TCGA-CD-8535-01A-11R-2343-13,TCGA-VQ-A8PK-01A-12R-A414-31,TCGA-VQ-AA6D-01A-11R-A414-31,TCGA-CG-5721-01A-11R-1602-13,TCGA-FP-8099-01A-11R-2343-13,TCGA-BR-A4J5-01A-21R-A251-31,...,TCGA-VQ-A91V-01A-11R-A414-31,TCGA-VQ-A91X-01A-12R-A414-31,TCGA-VQ-A923-01A-11R-A414-31,TCGA-EQ-8122-01A-11R-2343-13,TCGA-VQ-A8PO-01A-11R-A414-31,TCGA-B7-A5TK-01A-12R-A36D-31,TCGA-D7-8578-01A-21R-2343-13,TCGA-BR-A4J9-01A-12R-A251-31,TCGA-BR-4366-01A-01R-1157-13,TCGA-VQ-A8PH-01A-12R-A414-31
TSPAN6,3875,1352,751,3338,4901,1174,3838,1950,1943,1728,...,1184,32,1972,2813,4360,5816,2902,693,2560,6469
DPM1,2712,1849,1817,3823,6283,2242,4371,1825,3279,2406,...,2479,4732,3558,5673,2483,4611,2763,2393,12395,6627
SCYL3,1265,199,1046,1064,1445,885,1324,553,1136,1069,...,1649,1092,1330,1520,688,1041,768,1352,2650,624
FIRRM,501,264,467,607,1246,848,1034,385,804,366,...,895,765,727,1201,591,449,536,251,1540,707
FGR,644,263,557,344,154,230,206,1498,765,839,...,269,61,929,646,706,1219,489,366,379,181


In [9]:
df_filtr_novo.to_csv("../dados npo/DADOS PUBLICOS/expressao_filtrada.csv")